In [7]:
#!conda install -c anaconda beautifulsoup4 --yes

import bs4 as bs
import urllib.request
import pandas as pd

In [33]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
canpc_pd = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
canpc_pd.drop(canpc_pd[canpc_pd.Borough == "Not assigned"].index, inplace=True)
canpc_pd.drop(0, inplace=True)

canpc_pd['Neighborhood'] = canpc_pd['Neighborhood'].replace({'\\n':''}, regex=True)
canpc_pd['Neighborhood'] = canpc_pd['Neighborhood'].replace('Not assigned',canpc_pd['Borough'],regex= True)
canpc_pd.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [30]:
canpc_pd['Neighborhood'] = canpc_pd[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
canpc_pd[['PostalCode','Borough','Neighborhood']].drop_duplicates()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Harbourfront,Regent Park"
7,M6A,North York,"Lawrence Heights,Lawrence Manor"
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,"Rouge,Malvern"
15,M3B,North York,Don Mills North
16,M4B,East York,"Woodbine Gardens,Parkview Hill"
18,M5B,Downtown Toronto,"Ryerson,Garden District"


In [34]:
cangeo_pd = pd.read_csv('https://cocl.us/Geospatial_data')
cangeo_pd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
canall_pd = pd.merge(canpc_pd, cangeo_pd,  how='inner', left_on=['PostalCode'], right_on = ['Postal Code'])
canall_pd.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
